# कार्टपोल संतुलनासाठी RL प्रशिक्षण

हे नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) चा भाग आहे. हे [अधिकृत PyTorch ट्यूटोरियल](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) आणि [Cartpole PyTorch अंमलबजावणी](https://github.com/yc930401/Actor-Critic-pytorch) यापासून प्रेरित आहे.

या उदाहरणात, आपण RL वापरून एक मॉडेल प्रशिक्षित करू जे एका गाड्यावर असलेल्या खांबाला संतुलित ठेवेल, जो आडव्या पट्टीवर डावीकडे आणि उजवीकडे हलू शकतो. आम्ही [OpenAI Gym](https://www.gymlibrary.ml/) पर्यावरणाचा वापर करून खांबाचे अनुकरण करू.

> **Note**: तुम्ही या धड्याचा कोड स्थानिक पातळीवर (उदा. Visual Studio Code मधून) चालवू शकता, अशा परिस्थितीत अनुकरण एका नवीन विंडोमध्ये उघडेल. ऑनलाइन कोड चालवताना, तुम्हाला कोडमध्ये काही बदल करावे लागतील, जसे [येथे](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) वर्णन केले आहे.

आपण सुरुवात जिम स्थापित असल्याची खात्री करून करू:


In [ ]:
import sys
!{sys.executable} -m pip install gym

आता CartPole वातावरण तयार करूया आणि त्यावर कसे कार्य करायचे ते पाहूया. एका वातावरणामध्ये खालील गुणधर्म असतात:

* **Action space** म्हणजे प्रत्येक सिम्युलेशनच्या टप्प्यावर आपण करू शकणाऱ्या संभाव्य क्रियांची श्रेणी  
* **Observation space** म्हणजे आपण करू शकणाऱ्या निरीक्षणांची श्रेणी  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

चला पाहूया की सिम्युलेशन कसे कार्य करते. खालील लूप सिम्युलेशन चालवतो, जोपर्यंत `env.step` टर्मिनेशन फ्लॅग `done` परत करत नाही. आम्ही `env.action_space.sample()` वापरून कृती निवडू, ज्याचा अर्थ असा आहे की प्रयोग फार लवकर अपयशी होण्याची शक्यता आहे (CartPole वातावरण संपते जेव्हा CartPole चा वेग, त्याचे स्थान किंवा कोन विशिष्ट मर्यादांच्या बाहेर जातो).

> सिम्युलेशन नवीन विंडोमध्ये उघडेल. तुम्ही कोड अनेक वेळा चालवू शकता आणि त्याचे वर्तन कसे आहे ते पाहू शकता.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

आपण पाहू शकता की निरीक्षणांमध्ये 4 संख्या असतात. त्या आहेत:
- गाडीची स्थिती
- गाडीचा वेग
- खांबाचा कोन
- खांबाच्या फिरण्याचा दर

`rew` हा प्रत्येक टप्प्यावर आपल्याला मिळणारा बक्षीस आहे. CartPole वातावरणात, प्रत्येक सिम्युलेशन टप्प्यासाठी तुम्हाला 1 गुण दिला जातो, आणि उद्दिष्ट म्हणजे एकूण बक्षीस जास्तीत जास्त करणे, म्हणजे CartPole पडल्याशिवाय तोल सांभाळण्याचा कालावधी वाढवणे.

रिइनफोर्समेंट लर्निंग दरम्यान, आपले उद्दिष्ट म्हणजे एक **पॉलिसी** $\pi$ तयार करणे, जी प्रत्येक स्थिती $s$ साठी आपल्याला कोणती क्रिया $a$ घ्यायची ते सांगेल, म्हणजेच मूलतः $a = \pi(s)$.

जर तुम्हाला संभाव्यताधारित उपाय हवा असेल, तर तुम्ही पॉलिसीला प्रत्येक क्रियेसाठी संभाव्यता परत करणारी प्रणाली म्हणून विचार करू शकता, म्हणजे $\pi(a|s)$ याचा अर्थ असा होईल की स्थिती $s$ वर आपण क्रिया $a$ घ्यावी याची संभाव्यता.

## पॉलिसी ग्रेडियंट पद्धत

सर्वात सोप्या RL अल्गोरिदममध्ये, ज्याला **पॉलिसी ग्रेडियंट** म्हणतात, आपण पुढील क्रिया भाकीत करण्यासाठी एक न्यूरल नेटवर्क प्रशिक्षित करू.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

आम्ही अनेक प्रयोग चालवून आणि प्रत्येक प्रयोगानंतर आमच्या नेटवर्कचे अद्यतन करून नेटवर्क प्रशिक्षण देऊ. चला एक फंक्शन परिभाषित करूया जे प्रयोग चालवेल आणि परिणाम परत करेल (तथाकथित **ट्रेस**) - सर्व स्थिती, क्रिया (आणि त्यांचे शिफारस केलेले संभाव्यता), आणि बक्षिसे:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

आपण प्रशिक्षित न केलेल्या नेटवर्कसह एक एपिसोड चालवू शकता आणि लक्षात घेऊ शकता की एकूण बक्षीस (एपिसोडची लांबी म्हणून ओळखले जाते) खूप कमी आहे:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

धोरण ग्रेडियंट अल्गोरिदमच्या क्लिष्ट पैलूंमध्ये **सवलतीच्या बक्षिसांचा** वापर करणे आहे. कल्पना अशी आहे की आपण खेळाच्या प्रत्येक टप्प्यावर एकूण बक्षिसांचा वेक्टर गणना करतो, आणि या प्रक्रियेदरम्यान काही गुणांक $gamma$ वापरून सुरुवातीची बक्षिसे सवलत देतो. आम्ही परिणामी वेक्टर सामान्यीकृत देखील करतो, कारण आम्ही ते आमच्या प्रशिक्षणावर परिणाम करण्यासाठी वजन म्हणून वापरणार आहोत:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

आता प्रत्यक्ष प्रशिक्षण सुरू करूया! आपण 300 एपिसोड चालवणार आहोत, आणि प्रत्येक एपिसोडमध्ये खालील गोष्टी करूया:

1. प्रयोग चालवा आणि ट्रेस गोळा करा
1. घेतलेल्या कृती आणि अंदाजित संभाव्यता यांच्यातील फरक (`gradients`) मोजा. फरक जितका कमी असेल, तितके आम्हाला खात्री आहे की योग्य कृती घेतली आहे.
1. सवलतीच्या बक्षिसे मोजा आणि सवलतीच्या बक्षिसांद्वारे `gradients` गुणाकार करा - यामुळे उच्च बक्षिस असलेल्या टप्प्यांचा अंतिम निकालावर अधिक प्रभाव पडेल, तुलनेने कमी बक्षिस असलेल्या टप्प्यांपेक्षा.
1. आमच्या न्यूरल नेटवर्कसाठी अपेक्षित लक्ष्य कृती काही प्रमाणात चालवताना अंदाजित संभाव्यतेमधून घेतल्या जातील, आणि काही प्रमाणात गणितीय `gradients` मधून. आम्ही `alpha` पॅरामीटर वापरून ठरवू की `gradients` आणि बक्षिसे किती प्रमाणात विचारात घेतली जातील - याला reinforcement algorithm चा *learning rate* म्हणतात.
1. शेवटी, आम्ही आमच्या नेटवर्कला states आणि अपेक्षित कृतींवर प्रशिक्षण देतो, आणि प्रक्रिया पुन्हा सुरू करतो.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

आता परिणाम पाहण्यासाठी रेंडरिंगसह एपिसोड चालवूया:


In [ ]:
_ = run_episode(render=True)

आशा आहे की आता तुम्हाला दिसत असेल की खांब चांगल्या प्रकारे संतुलित राहू शकतो!

## अभिनेता-समालोचक मॉडेल

अभिनेता-समालोचक मॉडेल हे पॉलिसी ग्रेडियंट्सचे पुढील विकसित रूप आहे, ज्यामध्ये आम्ही धोरण आणि अंदाजित बक्षिसे शिकण्यासाठी एक न्यूरल नेटवर्क तयार करतो. या नेटवर्कला दोन आउटपुट्स असतील (किंवा तुम्ही याला दोन स्वतंत्र नेटवर्क्स म्हणून पाहू शकता):
* **अभिनेता (Actor)** पॉलिसी ग्रेडियंट मॉडेलप्रमाणेच, स्थितीची संभाव्यता वितरण देऊन कोणती कृती करायची याची शिफारस करतो.
* **समालोचक (Critic)** त्या कृतींमधून मिळणाऱ्या बक्षिसांचा अंदाज लावतो. दिलेल्या स्थितीत भविष्यातील एकूण अंदाजित बक्षिसे परत करतो.

चला असे मॉडेल परिभाषित करूया:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

आम्हाला आमच्या `discounted_rewards` आणि `run_episode` फंक्शन्समध्ये थोडा बदल करावा लागेल:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


आता आपण मुख्य प्रशिक्षण लूप चालवू. योग्य तोटा फंक्शन्स मोजून आणि नेटवर्क पॅरामीटर्स अद्यतनित करून आपण मॅन्युअल नेटवर्क प्रशिक्षण प्रक्रिया वापरू:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## मुख्य मुद्दा

या डेमोमध्ये आपण दोन RL अल्गोरिदम पाहिले: साधा पॉलिसी ग्रेडियंट आणि अधिक प्रगत अ‍ॅक्टर-क्रिटिक. तुम्ही पाहू शकता की हे अल्गोरिदम स्टेट, अ‍ॅक्शन आणि रिवॉर्ड यांसारख्या अमूर्त संकल्पनांवर कार्य करतात - त्यामुळे ते खूप वेगवेगळ्या वातावरणांमध्ये लागू केले जाऊ शकतात.

रिइनफोर्समेंट लर्निंग आपल्याला केवळ अंतिम रिवॉर्ड पाहून समस्या सोडवण्यासाठी सर्वोत्तम रणनीती शिकण्याची परवानगी देते. लेबल केलेल्या डेटासेटची गरज नसल्यामुळे, आपल्याला आपल्या मॉडेल्सचा ऑप्टिमायझेशन करण्यासाठी सिम्युलेशन्स अनेक वेळा पुन्हा पुन्हा चालवता येतात. तथापि, RL मध्ये अजूनही अनेक आव्हाने आहेत, जी तुम्हाला शिकता येतील जर तुम्ही या AI च्या रंजक क्षेत्रावर अधिक लक्ष केंद्रित करण्याचा निर्णय घेतला.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
